In [ ]:
import matplotlib.pyplot as plt
import panel as pn

pn.extension(template="fast")
pn.config.comms = "ipywidgets"
import xarray as xr

import cfspopcon
from cfspopcon.unit_handling import ureg
from cfspopcon.named_options import ConfinementScaling, Impurity
import yaml
from typing import List
import git
from bokeh.models.formatters import PrintfTickFormatter


PROJECT_NAME = "BARC"
PROJECT_DIR = f"../../example_cases/{PROJECT_NAME}"
input_parameters, algorithm, points = cfspopcon.read_case(PROJECT_DIR)
plot_style = cfspopcon.read_plot_style(f"{PROJECT_DIR}/plot_popcon.yaml")
with open(f"{PROJECT_DIR}/gui.yaml", "r") as file:
    gui_config = yaml.safe_load(file)

algorithm.validate_inputs(input_parameters)
dataset = xr.Dataset(input_parameters)
repo = git.Repo(search_parent_directories=True)
sha = repo.head.commit.hexsha
short_sha = repo.git.rev_parse(sha, short=True)

In [ ]:
confinement_names = ConfinementScaling._member_map_.keys()
select_confinement = pn.widgets.Select(name="Confinement Scaling", options=list(confinement_names)).servable(target="sidebar")

In [ ]:
def build_sliders(gui_config: dict, target: str) -> List[pn.widgets.FloatSlider]:
    sliders = {}
    units = {}
    for slide_conf in gui_config["sliders"]:
        dataset[slide_conf["name"]] = dataset[slide_conf["name"]].astype(float)
        slider = pn.widgets.FloatSlider(
            name=slide_conf["name"],
            start=slide_conf["start"],
            end=slide_conf["end"],
            value=float(dataset[slide_conf["name"]].pint.to(slide_conf["unit"]).values),
            step=slide_conf["step_size"],
        ).servable(target=target)
        sliders[slide_conf["name"]] = slider.param.value_throttled
        units[slide_conf["name"]] = slide_conf["unit"]
    return sliders, units

impurity_names = [imp.name for imp in input_parameters['impurities'].dim_species.values]

def build_impurity_sliders(gui_config: dict):
    def build_impurity_slider(imp_config: dict):
        imp_name = imp_config["name"]
        initial_value = dataset['impurities'].sel(dim_species=Impurity._member_map_[imp_name]).values
        slider = pn.widgets.FloatSlider(name=imp_name,
                                        start=float(imp_config["start"]), 
                                        end=float(imp_config["end"]), 
                                        step=float(imp_config["step_size"]), 
                                        value=float(initial_value), 
                                        format=PrintfTickFormatter(format="%.2e")).servable(target="sidebar")
        return slider.param.value_throttled
    
    sliders = {}
    units = {}
    for imp_config in gui_config["impurity_concentration_sliders"]:
        imp_name = imp_config["name"]
        sliders[imp_name] = build_impurity_slider(imp_config)
        units[imp_name] = ""
    return sliders, units

sliders, units = build_sliders(gui_config, "sidebar")
imp_sliders, imp_units = build_impurity_sliders(gui_config)
sliders = {**sliders, **imp_sliders}
units = {**units, **imp_units}

In [ ]:
_, ax = plt.subplots(figsize=plot_style["figsize"], dpi=plot_style["show_dpi"])


def make_plot(select_confinement: str, **kwargs):
    dataset["energy_confinement_scaling"] = ConfinementScaling._member_map_[select_confinement]
    for key, value in kwargs.items():
        if key in imp_sliders.keys():
            key_value = Impurity._member_map_[key]
            dataset['impurities'].loc[dict(dim_species=key_value)] = value * ureg(units[key])
        else:
            dataset[key] = value * ureg(units[key])
    algorithm.update_dataset(dataset, in_place=True)
    fig, ax = cfspopcon.plotting.make_plot(dataset, plot_style, points={}, title=f"{PROJECT_NAME} (commit: {short_sha})", output_dir=None)
    return pn.pane.Matplotlib(fig)


interactive_plot = pn.param.ParamFunction(pn.bind(make_plot, select_confinement=select_confinement, **sliders), loading_indicator=True)
app = pn.Column(interactive_plot)
app.servable(target="main")